# ひき算
量子コンピュータには従来のコンピュータと同じ計算もできるという特徴があります。ひき算の回路を確認します。

## 今回学ぶこと
1. 量子ゲートを使って二進数のひき算を実装
2. 量子の重ね合わせを使って1つの回路で複数のひき算

## Blueqatのインストール
pipからBlueqatをインストールします。

In [2]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## 二進数のひき算
たし算は符号を判定するccxゲートと、位の足し合わせを実行するcxゲートを使います。今回はa-b=cdという二進数のひき算の量子回路を行います。今回はaとbの値によって4種類のひき算を実装します。それぞれのひき算は、

0-0 = 00 => 0000  
0-1 = 11 => 0111  
1-0 = 01 => 1001  
1-1 = 00 => 1100  

となります。4量子ビットのビット列で表現し、前半の2量子ビットが入力値aとbで、後半の2量子ビットが出力値cとdです。aとbを入力する回路と、実際に引き算を実現する回路を別に作り、何度か使い回します。1のようなデータの入力はXゲートを使って0を反転させて実装します。

ひき算の回路部分はこのようになります。*はコントロールビットです。

```
a ---X---*---X---*------- a
b -------*-------|---*--- b
0 -------X-------|---|--- c
0 ---------------X---X--- d
```

aとbにXゲートを使ってデータを入れると、ccx回路で符号の判定、cx回路で位の引き算を行います。

In [1]:
#ツールの読み込み
from blueqat import Circuit

#ひき算部分を実現します。
substractor = Circuit().x[0].ccx[0,1,2].x[0].cx[0,3].cx[1,3]

In [2]:
#0-0
(Circuit() + substractor).m[:].run(shots=100)

Counter({'0000': 100})

In [3]:
#0-1
(Circuit().x[1] + substractor).m[:].run(shots=100)

Counter({'0111': 100})

In [5]:
#1-0
(Circuit().x[0] + substractor).m[:].run(shots=100)

Counter({'1001': 100})

In [6]:
#1-1
(Circuit().x[0,1] + substractor).m[:].run(shots=100)

Counter({'1100': 100})

このように、ひき算が実装できました。

## 重ね合わせを利用したひき算
ここで、Xゲートでデータを一つ一つ入れる代わりに、Hゲートを使ってひき算をしてみます。

In [7]:
#Xゲートの代わりにHゲートを入力に使う
(Circuit().h[0,1] + substractor).m[:].run(shots=100)

Counter({'0000': 25, '1100': 23, '1001': 25, '0111': 27})

アダマールゲートを利用すると4つのひき算が大体1/4ずつ答えとして出てきました。このように汎用ひき算回路を作ると、重ね合わせ状態を利用した計算を実行できます。

## もつれを利用したひき算
次にHゲートの代わりにa-b=0となるたし算を量子のもつれを使って行ってみます。

In [8]:
#00と11のもつれを作る
(Circuit().h[0].cx[0,1] + substractor).m[:].run(shots=100)

Counter({'1100': 51, '0000': 49})

このように、入力値が00と11がもつれているので、この2つのひき算が約1/2ずつ出てきます。

--------

## 解説：回路の作成（符号）
まずは上の符号から作成します。
符号は、0, 1, 0, 0 の順になっています。

表で表すと以下になります。

|X|Y|符号|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|0|
|1|1|0|

次にXの桁を反転させると

|X|Y|符号|
|:-:|:-:|:-:|
|1|0|0|
|1|1|1|
|0|0|0|
|0|1|0|

これを見ると CCXゲートと同じことがわかります。
(CCXは入力の頭2つのビットが1ならば3番目のビットを反転させる)

このことから入力が 11 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_0.png" width="23%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（２桁目）
次にもう一つのくらいを作成します。
2桁目を見ると、0, 1, 1, 0 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの1桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

これを見ると CXゲートと同じことがわかります。
(左辺がCXの入力で右辺が出力の2番目のビット)

このことから入力が 10 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_1.png" width="27%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（全体）
最後に上の二つの回路をまとめます。

入力を 00 としてまとめた回路を以下に示します。

<img src="./img/008/008_02_3.png" width="40%">

上2ビットを入力、残りの2ビットが出力です。
始めの CCX は符号の部分で残りの 2つの CX は引き算の部分です。   
符号を検知するためにXゲートを施したのでCCXした後は再びXをして戻しています。

## 解説：重ね合わせ回路の実装
量子の重ね合わせを用いると4パターンを一気に操作することができます。

式としては以下の物を考えます。

<img src="./img/009/009_02_0.png" width="35%">

見てわかる通り右辺に入力の4パターンが出てきました。
この性質を用いて実装します。

回路は以下のようになります。

<img src="./img/009/009_02_2.png" width="40%">

今まで Xゲートを施していた部分を Hゲートにして 4パターンを作っています。